In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
import scipy.misc
import os
from environment import SM_env
from environment import random_normal_trunc
from environment import eth_env
from environment import SM_env_with_stale
from environment import random_normal_trunc
import mdptoolbox
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(allow_growth=True)

/home/jvy5516/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jvy5516/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jvy5516/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jvy5516/miniconda3/envs/py37/lib/python3

In [2]:
class Qnetwork():
    def __init__(self, h_size, state_space_n, state_vector_n, action_space_n):

        self.state_space_n = state_space_n
        self.action_space_n = action_space_n
        self.state_vector_n = state_vector_n

        # The network recieves a state number from
        # It then resizes it and processes it through four convolutional layers.
        self.vectorIn = tf.placeholder(shape=[None, state_vector_n], dtype=tf.float32)
        #print(self.scalarInput)
        #self.vectorIn = tf.one_hot(self.scalarInput, state_space_n, dtype=tf.float32)
        #print(self.vectorIn)
        self.fc1 = tf.layers.dense(self.vectorIn, h_size, activation=tf.nn.relu)
        #print(self.fc1)
        self.fc2 = tf.layers.dense(self.fc1, h_size, activation=tf.nn.relu)
        #print(self.fc2)

        '''
        self.imageIn = tf.reshape(self.scalarInput, shape=[-1, 84, 84, 3])
        self.conv1 = slim.conv2d( \
            inputs=self.imageIn, num_outputs=32, kernel_size=[8, 8], stride=[4, 4], padding='VALID',
            biases_initializer=None)
        self.conv2 = slim.conv2d( \
            inputs=self.conv1, num_outputs=64, kernel_size=[4, 4], stride=[2, 2], padding='VALID',
            biases_initializer=None)
        self.conv3 = slim.conv2d( \
            inputs=self.conv2, num_outputs=64, kernel_size=[3, 3], stride=[1, 1], padding='VALID',
            biases_initializer=None)
        self.conv4 = slim.conv2d( \
            inputs=self.conv3, num_outputs=h_size, kernel_size=[7, 7], stride=[1, 1], padding='VALID',
            biases_initializer=None)
        '''

        # We take the output from the final layer and split it into separate advantage and value streams.
        #self.streamAC, self.streamVC = tf.split(self.conv4, 2, 3)
        #self.streamA = slim.flatten(self.streamAC)
        #self.streamV = slim.flatten(self.streamVC)

        #print(self.fc2)

        self.streamA, self.streamV = tf.split(self.fc2, 2, 1)
        xavier_init = tf.contrib.layers.xavier_initializer()
        self.AW = tf.Variable(xavier_init([h_size // 2, action_space_n]))
        self.VW = tf.Variable(xavier_init([h_size // 2, 1]))
        self.Advantage = tf.matmul(self.streamA, self.AW)
        self.Value = tf.matmul(self.streamV, self.VW)
        # Then combine them together to get our final Q-values.
        self.Qout = self.Value + tf.subtract(self.Advantage, tf.reduce_mean(self.Advantage, axis=1, keep_dims=True))
        #print(self.Qout)
        self.predict = tf.argmax(self.Qout, 1)

        # Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
        self.targetQ = tf.placeholder(shape=[None], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions, action_space_n, dtype=tf.float32)

        self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)

        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

    def get_Q_table(self, sess, s):
        Q = sess.run(self.Qout, feed_dict={self.vectorIn:[s]})
        Q = np.reshape(Q, [-1])
        return Q

    def act_epsilon_greedy(self, sess, s, e = 0):

        #legal_move_list = env.legal_move_list(s)
        #legal_move_list = range(env._action_space_n)

        if np.random.rand(1) < e:
            a = np.random.choice(self.action_space_n)
        else:
            Q = self.get_Q_table(sess, s)
            a = np.argmax(Q)

            '''
            #print(Q)
            a = 0
            val = -100000
            for i in range(self.):
                if (Q[i] > val):
                    val = Q[i]
                    a = i
            '''

        return a

    def get_policy_table(self, sess, env):
        policy = np.zeros(self.state_space_n, dtype = np.int32)
        for i in range(0, self.state_space_n):
            ss = env._index_to_vector(i)
            policy[i] = self.act_epsilon_greedy(sess, ss, 0)
        return policy

HIDDEN_BLOCK = 20 # maximum hidden block of attacker
rule = "longest" # "longest" -- bitcoin rule, "GHOST" -- GHOST rule
h_size = 100 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
path = "./btc_" + rule + str(HIDDEN_BLOCK) + "_" + str(h_size) #The path to save our model to.
know_alpha = True # if the agent knows the current alpha.
# if know_alpha == True: path += "know_alpha"
best_path = path + "_cp/model_best.ckpt" # best model path

stale_rate = 0.0 # stale block rate, 0 means no stale block -- the classical selfish mining setting
ALPHA = 0.4 # the hash power fraction of attacker
GAMMA = 0.5 # the follower's fraction
DEV = 0.0 # the alpha's fluctuation rate, 0 means fixed alpha
know_alpha = True # if the agent knows the current alpha.
random_process = "iid" # or "brown" -- brownian process
interval = (0, 0.5) # the range of the alpha

env = SM_env_with_stale(max_hidden_block = HIDDEN_BLOCK, attacker_fraction = ALPHA, follower_fraction = GAMMA, rule = rule, stale_rate=stale_rate, dev = DEV, know_alpha = know_alpha, random_interval=interval, random_process = random_process, frequency=6)

h_size = 100 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
load_best_model = True # load a saved best model
init = tf.global_variables_initializer()
mainQN = Qnetwork(h_size, env._state_space_n, env._state_vector_n, env._action_space_n)
targetQN = Qnetwork(h_size, env._state_space_n, env._state_vector_n, env._action_space_n)
saver = tf.train.Saver()


state space size =  733


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [3]:
num_tra = 100
seed_trial = 5
index_file = 'episode_15/train_index.txt'
index_pd = pd.read_table(index_file, sep=';', header=None)
index_loc = index_pd[0].values

In [4]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    if load_best_model == True:
        print('Loading Best Model..')
        saver.restore(sess, best_path)

    reward_origin = []
    for index in range(num_tra):
        env.seed(index_loc[index])
        s = env.reset()
        s_0 = s
        r_total = 0
        for i in range(100):
            a = mainQN.act_epsilon_greedy(sess, s, 0)
            s, r, d, _ = env.step(s, a, move = True)
            r_total += r
            if s == s_0:
                break
        reward_origin.append(r_total)
    sess.close()
    



Loading Best Model..
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt


In [7]:
from random import choice
def random_run(choice_state,reward_origin):
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        if load_best_model == True:
            print('Loading Best Model..')
            saver.restore(sess, best_path)

        reward_random = []
        for index in range(num_tra):
            # print(index)
            break_state = choice_state[index] - 1
            # break_state = 180
            flag = False
            average = []
            for t in range(seed_trial):
                env.seed(index_loc[index])
                s = env.reset()
                s_0 = s
                if flag:
                    continue
                r_total = 0
                modifiable = False
                for i in range(200):
                    a = mainQN.act_epsilon_greedy(sess, s, 0)
                    if (i==break_state):
                        modifiable = True
                        legal_state = env.legal_move_list(s)
                        if a not in legal_state:
                            a = legal_state[0]
                        legal_state.remove(a)
                        if len(legal_state)==0:
                            flag = True
                            break
                        elif len(legal_state)==1:
                            a = legal_state[0]
                        elif len(legal_state)==2:
                            a = choice(legal_state)
                        # print("The action is: ", a)
                    s, r, d, _ = env.step(s, a, move = True)
                    r_total+=r
                    # line = ("The block index is " + str(i) + "         "+ str(s[0]) + ";" + str(s[1]) + ";" + str(s[3]) + ";")
                    # print(line)
                    if flag==False and s == s_0:
                        break
                    
                    # if (i==break_state):
                    #     env.seed(2021+index+100+t)
                if flag==False and modifiable==True:
                    average.append(abs(r_total-reward_origin[index]))

            if len(average)==0:
                r=-1
            else:
                r = sum(average) / len(average)
                # r = max(average)
            reward_random.append(r)
        sess.close()
    return reward_random

def read_loc(method):
    summary = 'summary_explain/' + method + '.txt'
    summary_pd = pd.read_table(summary, sep=';', header=None)
    return summary_pd[0].values

def process(loc,num,reward_origin,thres=100):

    dif = random_run(loc,reward_origin)
    sum = 0
    num = 0
    for i in range(len(dif)):
        if 0<=dif[i]<thres:
            sum+=dif[i]
            num+=1
    return sum/num

In [8]:
con_loc = read_loc('concept')
att_loc = read_loc('attention')
the_loc = read_loc('theta')
sal_loc = read_loc('saliency')
smo_loc = read_loc('smooth')
int_loc = read_loc('integrated')
ran_loc = np.random.randint(5,10,size=[num_tra])

con_diff = process(con_loc,num_tra,reward_origin)
att_diff = process(att_loc,num_tra,reward_origin)
the_diff = process(the_loc,num_tra,reward_origin)
sal_diff = process(sal_loc,num_tra,reward_origin)
smo_diff = process(smo_loc,num_tra,reward_origin)
int_diff = process(int_loc,num_tra,reward_origin)
ran_diff = process(ran_loc,num_tra,reward_origin)


print(con_diff,att_diff,the_diff,sal_diff,smo_diff,int_diff,ran_diff)

Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
Loading Best Model..
INFO:tensorflow:Restoring parameters from ./btc_longest20_100_cp/model_best.ckpt
1.6777600979192167 0.9497868217054263 1.4417955651703622 1.3581536293164196 1.2528922371437927 1.3197219994653835 0.9092343872484973
